In [1]:
from load_data import load_dataframe
df_train, df_test = load_dataframe()
df_train.head()

,class,fname,labels
0,13,00001.jpg,Audi TTS Coupe 2012
1,2,00002.jpg,Acura TL Sedan 2012
2,90,00003.jpg,Dodge Dakota Club Cab 2007
3,133,00004.jpg,Hyundai Sonata Hybrid Sedan 2012
4,105,00005.jpg,Ford F-450 Super Duty Crew Cab 2012


In [2]:
from load_data import create_generators
train_generator, validation_generator, test_generator = create_generators()

Found 6516 images.
Found 1628 images.
Found 8041 images.


### Load models

In [3]:
import tensorflow as tf

In [4]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_top=False,
)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### Add Layers on top of model

In [7]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(196, activation='sigmoid')
])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 196)               401604    
Total params: 23,989,316
Trainable params: 401,604
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
optimizer = tf.keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

### Train models

In [ ]:
epochs = 10
batch_size = 32
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator, 
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator
                             )